**<h1 align="center"><u>Insights and Exploratory Data Analysis(EDA) on Novel Corona Virus 2020</u></h1>**

In [ ]:
#Import all the required libraries
#Graphical Libraries
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from folium.plugins import MarkerCluster, MiniMap, Fullscreen
import branca
from IPython.display import IFrame, YouTubeVideo

#Manipulation
from datetime import date
import pandas as pd
import numpy as np

#Kaggle default
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#To supress warnings
import warnings
warnings.filterwarnings('ignore')

#To get the geolocation details
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

**<h2><u>Coronaviruses (CoV):</u></h2>**
"Coronaviruses (CoV) are a large family of viruses that cause illness ranging from the common cold to more severe diseases. Some coronaviruses transmit between animals, some between animals and people, and others from people to people." [[1].(https://www.canada.ca/en/public-health/services/diseases/coronavirus.html)]

The Wall Street Journal has also released an informative article [[2](https://www.wsj.com/articles/what-we-know-about-the-wuhan-virus-11579716128)] on the virus. More information about the virus can be found at the WHO [[3](https://www.who.int/emergencies/diseases/novel-coronavirus-2019)] website. WHO has also published a video [[4](https://youtu.be/mOV1aBVYKGA)] on youtube to make people aware of the health emergency caused by the novel Coronavirus. The video is given below:

In [ ]:
from IPython.display import IFrame, YouTubeVideo
YouTubeVideo('mOV1aBVYKGA',width=600, height=400)

**<h2><u>Purpose of this notebook:</u></h2>**

The purpose of this notebook is to provide insights into the data [[5](https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset)] scrapped from a dashboard [[6](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)] created by Johns Hopkins University. The data is made available in csv format by SRK [[7](https://www.kaggle.com/sudalairajkumar)].

Please note that the virus and information available on it is relatively new which means that the information available now might change in the future [[4](https://youtu.be/mOV1aBVYKGA)]. It also implies that the insights and observations drafted in this notebook might change based on the changing dataset.

In [ ]:
#Import the dataset
df=pd.read_csv("../input/novel-corona-virus-2019-dataset/2019_nCoV_data.csv")

#Verify the first five rows for the sanity check
df.head()

In [ ]:
#Verify the shape of the data
print("Shape of dataframe: ", df.shape)

In [ ]:
#Convert the columns from float to int, and respective date columns for further analysis
df = df.astype({'Confirmed': 'int32', 'Deaths': 'int32', 
                'Recovered': 'int32', 'Last Update': 'datetime64',
                'Date': 'datetime64'})
df['Country'] = df['Country'].replace({'Mainland China': 'China'})

In [ ]:
#Get the data of the latest date from the dataset
maxDate = max(df['Date'])
df_lastDate = df[df['Date'] >  pd.Timestamp(date(maxDate.year,maxDate.month,maxDate.day))]

**<h3><u>Total number of Corona Virus cases worldwide:</u></h3>**

In [ ]:
#Print the total number of observations on cases -Worldwide
print('\033[1mTotal Confirmed cases worldwide: ',df_lastDate.Confirmed.sum())
print('\033[1mTotal Death cases worldwide: ',df_lastDate.Deaths.sum())
print('\033[1mTotal Recovered cases worldwide: ',df_lastDate.Recovered.sum())

**<h2><u>Analysis 1: Total number of Confirmed, Death and Recovered cases in each country</u></h2>**

To view this information, the data is first processed and then displayed on an interactive table created using plotly [[8](https://plot.ly/)]. The table is scrollable and the columns can be rearranged in any order.

In [ ]:
#Process data for each country
df_tempC = df_lastDate.groupby('Country').Confirmed.sum().to_frame()
df_tempD = df_lastDate.groupby('Country').Deaths.sum().to_frame()
df_tempR = df_lastDate.groupby('Country').Recovered.sum().to_frame()

#Merge the above data frames into one for convenient processing
df_temp = pd.merge(df_tempC, df_tempD, how='inner', left_index=True, right_index=True)
df_temp = pd.merge(df_temp, df_tempR, how='inner', left_index=True, right_index=True)
df_temp = df_temp.sort_values(['Confirmed'],ascending=[False])


#Create an interactive table based and fill the final data frame values
fig = go.Figure(data=[go.Table(header=dict(values=['<b>Country</b>','<b>Confirmed Cases</b>',
                                                   '<b>Death Cases</b>', '<b>Recovered Cases</b>'],
                                            fill_color='paleturquoise', 
                                            align=['left','center'],
                                           font=dict(color='black', size=16),
                                           height=40),
                               cells=dict(values=[df_temp.index, 
                                                  df_temp.Confirmed.values, df_temp.Deaths.values,
                                                 df_temp.Recovered.values,],
                                          fill_color='lavender', 
                                          align=['left','center'],
                                          font=dict(color='black', size=14),
                                         height=23)

                                )
                     ]
               )

#Cosmetic changes
fig.update_layout(title={'text':'<b>Number of Confirmed, Death and Recovered cases in each country</b>',
                        'y':0.92,
                        'x':0.5,
                        'xanchor':'center',
                        'yanchor':'top',
                        'font':dict(size=22)
                        }
                 )
fig.update_layout(height=600)

In [ ]:
print("\033[1mTotal number of countries affected:", len(df_temp.Confirmed.values))

> **<h3><u>Observations based on Analysis 1:</u></h3>**
> 
> 1. Most of the cases are found in China.
> 2. Few other countries where the high number of cases observed are Thailand, Singapore and Japan.
> 3. The total number of countries affected by the virus is 28.
> 4. The number of recovered cases is much lower relative to the confirmed cases.

**<h2><u>Analysis 2: Comparision of cases in China with Rest of the World</u></h2>**

To view this information, the data is processed and then displayed by interactive bar charts created using plotly [[8](https://plot.ly/)]. The exact number of cases can be viewed by mouse hover on each bar. Though the purpose of the bar chart is to compare the cases in China with the Rest of the World, however, the legend is selectable. So, by clicking on any of the legend items, the bar charts would show data specific to the selected legend item. As the number of confirmed cases in China are more, so bars having lower counts can also be selected to view data only specific to the selection.

In [ ]:
#Process cases of China and Rest of the World
chinaConfirmed = df_lastDate[df_lastDate.Country=='China'].Confirmed.sum()
notChinaConfirmed = df_lastDate[df_lastDate.Country!='China'].Confirmed.sum()
chinaDeaths = df_lastDate[df_lastDate.Country=='China'].Deaths.sum()
notChinaDeaths = df_lastDate[df_lastDate.Country!='China'].Deaths.sum()
chinaRecovered = df_lastDate[df_lastDate.Country=='China'].Recovered.sum()
notChinaRecovered = df_lastDate[df_lastDate.Country!='China'].Recovered.sum()

#yAxis labels for the figure
yAxisChina = [chinaConfirmed, chinaDeaths, chinaRecovered]
yAxisNotChina = [notChinaConfirmed,notChinaDeaths,notChinaRecovered]

x=['Confirmed', 'Death', 'Recovered']
fig = go.Figure(go.Bar(x=x, y=[chinaConfirmed, chinaDeaths, chinaRecovered],text=yAxisChina, textposition='outside',
                       hovertemplate = "%{x}: %{y} </br>", name='China', marker_color='rgb(55, 83, 109)'))
fig.add_trace(go.Bar(x=x, y=[notChinaConfirmed,notChinaDeaths,notChinaRecovered],text=yAxisNotChina, textposition='outside', 
                     hovertemplate = "%{x}: %{y} </br>", name='Rest of the World',marker_color='rgb(26, 118, 255)'))

fig.update_layout(barmode='group', xaxis={'categoryorder':'category ascending'})
fig.update_layout(
    title={'text':'<b>Number of Confirmed, Death and Recovered cases in China and Rest of the World</b>',
                       'x':0.1,'xanchor':'left','font':dict(size=20,color='black')},
    xaxis_tickfont_size=14,
    legend=dict(
        x=1,
        y=1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15 # gap between bars of adjacent location coordinates.
#     bargroupgap=0.1 # gap between bars of the same location coordinate.
)

# Update xaxis properties
fig.update_xaxes(title_text="Type of Cases",  titlefont_size=16, tickfont_size=15)

# Update yaxis properties
fig.update_yaxes(title_text="Number of Cases", titlefont_size=16, tickfont_size=15)

fig.show()

> **<h3><u>Observations based on Analysis 2:</u></h3>**
> 
> 1. The number of cases in China are extremely high as compared to the rest of the world.
> 2. Due to the lack of much information on the virus, there is no vaccine to prevent it [[9](https://www.cdc.gov/coronavirus/2019-ncov/about/prevention-treatment.html)]. Hence, the recovery rate is low in China as well as in the rest of the world.
> 3. Though the number of deaths are comparatively lower than the number of confirmed cases worldwide yet the target is to control any further rise in the death count.
> 4. Despite global fears, the cases related to this virus are concentrated in China.

**<h2><u>Analysis 3: A geographical analysis of Confirmed cases with the Death cases worldwide</u></h2>**

To view this information, the data is processed and then displayed by an interactive map created using folium [[10](https://python-visualization.github.io/folium/)]. A legend is also created to provide a precise understanding of the markers on the map. The exact number of cases can be viewed by clicking on each marker. The map can also be viewed in fullscreen mode by selecting the option given on the top right side of the map.

In [ ]:
#Process and Merge the location details
df_lastDate['Province/State Copy'] = df_lastDate['Province/State'].fillna(' ')

df_lastDate['fullAddress'] = np.where((df_lastDate['Province/State Copy']==' ') |
                                         (df_lastDate['Province/State Copy']==df_lastDate['Country']), 
                                      df_lastDate['Country'], 
                                      df_lastDate['Province/State Copy'] + ', ' + df_lastDate['Country'])

In [ ]:
locator = Nominatim(user_agent="myGeocoder")
#put a delay of 1 second and fetch the geolocation details of each location
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df_lastDate['location'] = df_lastDate['fullAddress'].apply(geocode)
df_lastDate['point'] = df_lastDate['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df_lastDate[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_lastDate['point'].tolist(), index=df_lastDate.index)

In [ ]:
#Initialize map
intializeMap = folium.Figure(height=500)
custom_map = folium.Map(location=[42, 12], zoom_start=2, tiles='cartodbpositron').add_to(intializeMap)

fullscreen= Fullscreen(position='topright', 
                       title='Fullscreen', 
                       title_cancel='Exit Fullscreen', 
                       force_separate_button=True
                      ).add_to(custom_map)

#Create a custom html to show legend on map
legend_html = '''
                    {% macro html(this, kwargs) %}
                    <div style="
                        position: fixed; 
                        bottom: 50px;
                        left: 50px;
                        width: 250px;
                        height: 80px;
                        z-index:9999;
                        font-size:14px;
                        ">
                        <p>&emsp;     <i class="fa fa-map-marker fa-2x" style="color:darkblue"></i> Confirmed cases</p>
                        <p>&emsp;     <i class="fa fa-map-marker fa-2x" style="color:red"></i> Confirmed Deaths cases</p>
                    </div>
                    <div style="
                        position: fixed; 
                        bottom: 50px;
                        left: 50px;
                        width: 150px;
                        height: 80px; 
                        z-index:9998;
                        font-size:14px;
                        background-color: #ffffff;
                        filter: blur(8px);
                        -webkit-filter: blur(8px);
                        opacity: 0.7;
                        ">
                    </div>
                    {% endmacro %}
                '''
legend = branca.element.MacroElement()
legend._template = branca.element.Template(legend_html)
custom_map.get_root().add_child(legend)

#Add locations and styling on map
df_lastDate.apply(lambda row: folium.Marker(location=[row["latitude"], row["longitude"]]
                                                , popup = ('<b>Province/Country:</b> ' + row["fullAddress"]
                                                          +'<br>'+'<b>Confirmed:</b> ' + str(row['Confirmed'])
                                                          +'<br>'+'<b>Deaths:</b> ' + str(row['Deaths'])
                                                          +'<br>'+'<b>Recovered:</b> ' + str(row['Recovered'])
                                                          )
                                                , icon=folium.Icon(color='darkblue',icon='info-sign')
                                                  ,color='rgb(55, 83, 109)',fill_color='rgb(55, 83, 109)'
                                                ).add_to(custom_map) if (row['Deaths']==0) else
                  folium.Marker(location=[row["latitude"], row["longitude"]]
                                                , popup = ('<b>Province/Country:</b> ' + row["fullAddress"]
                                                          +'<br>'+'<b>Confirmed:</b> ' + str(row['Confirmed'])
                                                          +'<br>'+'<b>Deaths:</b> ' + str(row['Deaths'])
                                                          +'<br>'+'<b>Recovered:</b> ' + str(row['Recovered'])
                                                          )
                                                  ,icon=folium.Icon(color='red', icon='info-sign')
                                                  ,color='rgb(26, 118, 255)',fill_color='rgb(26, 118, 255)'
                                                ).add_to(custom_map), axis=1)

custom_map

> **<h3><u>Observations based on Analysis 3:</u></h3>**
> 
> 1. The number of death cases is relatively lower than the confirmed cases globally.
> 2. Considering the diameter of the death cases on the map, currently, the number of death observations have been recorded in China and nearby countries.

**<h2><u>Analysis 4: A detailed view of cases within Chinese Provinces</u></h2>**

To view this information, the data is processed and then displayed by interactive horizontal bar charts created using plotly [[8](https://plot.ly/)]. The exact number of cases can be viewed by mouse hover on each bar. Though the purpose of the bar chart is to compare all three types of cases i.e. Confirmed, Death and Recovered, however, the legend is selectable. So, by clicking on any of the legend items, the bar charts would show data specific to the selected legend item. As the number of confirmed cases in China are more, so bars having lower counts can also be selected to view data only specific to the selection.

In [ ]:
#Process data of Chinese provinces
chinaConfirmed = df_lastDate[df_lastDate.Country=='China'].Confirmed[:10]
chinaDeath = df_lastDate[df_lastDate.Country=='China'].Deaths[:10]
chinaRecovered = df_lastDate[df_lastDate.Country=='China'].Recovered[:10]
chinaProvinceName = df_lastDate[df_lastDate.Country=='China']['Province/State'][:10]

#Initialize the figure and start adding the traces
#China Confirmed cases
fig = go.Figure()
fig.add_trace(go.Bar(
                    y=chinaProvinceName,
                    x=chinaConfirmed,
                    name='Confirmed',
                    hovertemplate = "%{x}: %{y} </br>",
                    orientation='h',
                    marker=dict(
                                color='yellow',
                                line=dict(color='yellow', width=3)
                                )
                    )
             )
#China Death cases
fig.add_trace(go.Bar(
                    y=chinaProvinceName,
                    x=chinaDeath,
                    name='Death',
                    hovertemplate = "%{x}: %{y} </br>",
                    orientation='h',
                    marker=dict(
                        color='red',
                        line=dict(color='red', width=3)
                                )
                    )
             )
#China Recovered cases
fig.add_trace(go.Bar(
                    y=chinaProvinceName,
                    x=chinaRecovered,
                    name='Recovered',
                    hovertemplate = "%{x}: %{y} </br>",
                    orientation='h',
                    marker=dict(
                        color='green',
                        line=dict(color='green', width=3)
                                )
                    )
             )

#Cosmetic changes to figure
fig.update_layout(
                    title={'text':'<b>Top 10 provinces of China having highest number of Corona Virus cases</b>',
                           'x':0.15,'xanchor':'left','font':dict(size=20,color='black')
                          }
                 )
fig.update_layout(legend_orientation="h", 
                  legend=dict(
                                x=0.25,
                                y=-0.2,
                                bgcolor='rgba(255, 255, 255, 0)',
                                bordercolor='red'
                             )
                 )
# Update xaxis properties
fig.update_xaxes(title_text="Number of cases",  titlefont_size=18, tickfont_size=15)

# Update yaxis properties
fig.update_yaxes(title_text="China Provinces", titlefont_size=18, tickfont_size=15)
fig.update_layout(barmode='stack', height=600)
fig.show()

> **<h3><u>Observations based on Analysis 4:</u></h3>**
> 
> 1. The highest number of all three cases i.e. Confirmed, Death and Recovered have been observed in Hubei, Wuhan
> 2. The other provinces where cases have been observed are: Zhejiang, Guangdong, Henan and Hunan.
>         * Distance between Wuhan and Zhejiang: 410 miles 
>         * Distance between Wuhan and Zhejiang: 619 miles
>         * Distance between Wuhan and Zhejiang: 307 miles
>         * Distance between Wuhan and Zhejiang: 309 miles
>     Considering the distance between other provinces and Wuhan, and the high number of cases in Wuhan supports the news all over the web that virus has started spreading in Wuhan [[11](https://jamanetwork.com/journals/jama/fullarticle/2760500)].
> 3. Selecting only Recovered and Death cases from the legend of the above visualization, it can also be observed that the death cases are only linked to Wuhan. In all other provinces, there are much higher recovered cases than the death cases.

> Note: All the distances are measured using Google Maps [[12](https://www.google.com/maps)].


**<h2><u>Analysis 5: A detailed view of cases within China based on each increasing day since Jan 22, 2020</u></h2>**

To view this information, the data is processed and then displayed by interactive scatter plot charts created using plotly [[8](https://plot.ly/)]. The exact number of cases can be viewed by mouse hover on each bar. By clicking on any of the legend items, the scatter plot would show data specific to the selected legend item. Data from different dates can also be selected to display relevant key points.

In [ ]:
#Process the data of China based grouped by date
chinaTimelineC = df[df['Country']=='China'].groupby(df['Date'].dt.date)['Confirmed'].sum()
chinaTimelineD = df[df['Country']=='China'].groupby(df['Date'].dt.date)['Deaths'].sum()
chinaTimelineR = df[df['Country']=='China'].groupby(df['Date'].dt.date)['Recovered'].sum()

#Create figure with subplots
fig = make_subplots(rows=1, cols=2,  vertical_spacing=0.1, subplot_titles=("Confirmed Cases", "Death and Recovered Cases"))

#China confirmed cases
fig.add_trace(go.Scatter(name='Confirmed Cases',
                        y=chinaTimelineC.values,
                        x=chinaTimelineC.index,
                        text=chinaTimelineC.values,
                        textposition="top center",
                        mode='lines+markers',
                        hovertemplate = "%{x}: %{y} </br>",
                        marker=dict(color='yellow', size=10, line=dict(color='rgb(55, 83, 109)', width=3)),
                        line=dict(color='rgb(55, 83, 109)', width=4)
                        ), row=1, col=1
             )

#China death cases
fig.add_trace(go.Scatter(name='Death Cases',
                        y=chinaTimelineD.values,
                        x=chinaTimelineD.index,
                        text=chinaTimelineD.values,
                        textposition="bottom right",
                        hovertemplate = "%{x}: %{y} </br>",   
                        mode='lines+markers',
                        marker=dict(color='red', size=10, line=dict(color='rgb(55, 83, 109)', width=3)),
                        line=dict(color='rgb(55, 83, 109)', width=4)
                        ), row=1, col=2
             )

#China recovered cases
fig.add_trace(go.Scatter(name='Recovered Cases',
                        y=chinaTimelineR.values,
                        x=chinaTimelineR.index,
                        text=chinaTimelineR.values,
                        textposition="bottom right",
                        hovertemplate = "%{x}: %{y} </br>",
                        mode='lines+markers',
                        marker=dict(color='rgb(0, 196, 0)', size=10, line=dict(color='rgb(55, 83, 109)', width=3)),
                        line=dict(color='rgb(55, 83, 109)', width=4)
                        ), row=1, col=2
             )

#Cosmetic changes to figure
fig.update_layout(
                    title={'text':'<b>Comparision of Confirmed with Death and Recovered cases in China</b>',
                            'x':0.2,'xanchor':'left','font':dict(size=20,color='black')
                          }
                 )

fig.update_layout(legend_orientation="h", 
                  legend=dict(
                                x=0.25,
                                y=-0.3,
                                bgcolor='rgba(255, 255, 255, 0)',
                                bordercolor='red'
                            )
                 )

# Update xaxis properties
fig.update_xaxes(title_text="Timeline",  titlefont_size=16, tickfont_size=15, row=1, col=1)
fig.update_xaxes(title_text="Timeline",  titlefont_size=16, tickfont_size=15, row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Number of Cases", titlefont_size=16, tickfont_size=15, row=1, col=1)
fig.update_yaxes(title_text="Number of Cases", titlefont_size=16, tickfont_size=15, row=1, col=2)
fig.show()

> **<h3><u>Observations based on Analysis 5:</u></h3>**
> 
> 1. There is an exponential growth in the Confirmed cases in China. 
> 2. With each passing day, around 3K to 4K Confirmed cases are being reported in China.
> 3. The number of Recovered and Death cases has been continuously increasing until Feb 1, 2020. However, by getting more information on the virus and better prevention and awareness mechanisms, there is an increase in number of recovered cases as compared to Death cases i.e. 890:490 by Feb 4, 2020.

**<h2><u>Analysis 6: A detailed view of cases Worldwide excluding China based on each increasing day since Jan 22, 2020</u></h2>**

To view this information, the data is processed and then displayed by interactive scatter plot charts created using plotly [[8](https://plot.ly/)]. The exact number of cases can be viewed by mouse hover on each bar. By clicking on any of the legend items, the scatter plot would show data specific to the selected legend item. Data from different dates can also be selected to display relevant key points.

In [ ]:
#Process the data of Rest of the world grouped by date
notChinaTimelineC = df[df['Country']!='China'].groupby(df['Date'].dt.date)['Confirmed'].sum()
notChinaTimelineD = df[df['Country']!='China'].groupby(df['Date'].dt.date)['Deaths'].sum()
notChinaTimelineR = df[df['Country']!='China'].groupby(df['Date'].dt.date)['Recovered'].sum()

#Create figure with subplots
fig = make_subplots(rows=1, cols=2,  vertical_spacing=0.1, subplot_titles=("Confirmed Cases", "Death and Recovered Cases"))

#Rest of the World confirmed cases
fig.add_trace(go.Scatter(name='Confirmed Cases',
                        y=notChinaTimelineC.values,
                        x=notChinaTimelineC.index,
                        text=notChinaTimelineC.values,
                        textposition="top center",
                        mode='lines+markers',
                        hovertemplate = "%{x}: %{y} </br>",
                        marker=dict(color='yellow', size=10, line=dict(color='rgb(55, 83, 109)', width=3)),
                        line=dict(color='rgb(26, 118, 255)', width=4)
                        ), row=1, col=1
             )

#Rest of the World death cases
fig.add_trace(go.Scatter(name='Death Cases',
                        y=notChinaTimelineD.values,
                        x=notChinaTimelineD.index,
                        text=notChinaTimelineD.values,
                        textposition="bottom right",
                        mode='lines+markers',
                        hovertemplate = "%{x}: %{y} </br>",
                        marker=dict(color='red', size=10, line=dict(color='rgb(55, 83, 109)', width=3)),
                        line=dict(color='rgb(26, 118, 255)', width=4)
                        ), row=1, col=2
             )

#Rest of the World recovered cases
fig.add_trace(go.Scatter(name='Recovered Cases',
                        y=notChinaTimelineR.values,
                        x=notChinaTimelineR.index,
                        text=notChinaTimelineR.values,
                        textposition="bottom right",
                        mode='lines+markers',
                        hovertemplate = "%{x}: %{y} </br>",
                        marker=dict(color='rgb(0, 196, 0)', size=10, line=dict(color='rgb(55, 83, 109)', width=3)),
                        line=dict(color='rgb(26, 118, 255)', width=4)
                        ), row=1, col=2
             )

#Cosmetic changes to figure
fig.update_layout(
                title={'text':'<b>Comparision of Confirmed  cases v/s Death and Recovered cases globally excluding China</b>',
                            'x':0.09,'xanchor':'left','font':dict(size=20,color='black')
                       }
                )
fig.update_layout(legend_orientation="h", 
                  legend=dict(
                                x=0.25,
                                y=-0.3,
                                bgcolor='rgba(255, 255, 255, 0)',
                                bordercolor='red'
                             )
                 )
# Update xaxis properties
fig.update_xaxes(title_text="Timeline",  titlefont_size=16, tickfont_size=15, row=1, col=1)
fig.update_xaxes(title_text="Timeline",  titlefont_size=16, tickfont_size=15, row=1, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Number of Cases", titlefont_size=16, tickfont_size=15, row=1, col=1)
fig.update_yaxes(title_text="Number of Cases", titlefont_size=16, tickfont_size=15, row=1, col=2)
fig.show()

> **<h3><u>Observations based on Analysis 6:</u></h3>**
> 
> 1. The number of Confirmed cases are increasing globally. The high peak value is observed on Feb 4, 2020. Is it a hint of this virus becoming a Pandemic[[13](https://www.who.int/csr/disease/swineflu/frequently_asked_questions/pandemic/en/)] in the near future? [[14](https://www.sciencemag.org/news/2020/02/beast-moving-very-fast-will-new-coronavirus-be-contained-or-go-pandemic)]. According to Dr. Sylvie Briand, director of the World Health Organization's Infectious Hazards Management Department, "We are not in a pandemic" [[15](https://www.cnn.com/2020/02/04/health/who-coronavirus-not-pandemic/index.html)]. 
> 2. The number of recovered cases have become constant since Feb 1, 2020, as opposed to China where the Recovered cases are consistently increasing. 
> 3. The first death outside China due to the Novel Corona virus was observed on Feb 1, 2020. The last death count on Feb 4, 2020 has increased to two.

**<h3><u>Future possible work:</u></h3>**

Implementation of Machine Learning models to provide more insights.

**<h3 align="center">If you find my kernel insightful or you have learned something new then please UPVOTE.</h3>**

**<h2><u>References:</u></h2>**

[1] Canada, P. (2020). Coronavirus infection: Symptoms and treatment - Canada.ca. [online] Canada.ca. Available at: https://www.canada.ca/en/public-health/services/diseases/coronavirus.html [Accessed 6 Feb. 2020].

[2] McKay, B. (2020). What to Know About the New Coronavirus. [online] WSJ. Available at: https://www.wsj.com/articles/what-we-know-about-the-wuhan-virus-11579716128 [Accessed 6 Feb. 2020].

[3] Who.int. (2020). Coronavirus. [online] Available at: https://www.who.int/emergencies/diseases/novel-coronavirus-2019 [Accessed 6 Feb. 2020].

[4] YouTube. (2020). Novel coronavirus (2019-nCoV). [online] Available at: https://youtu.be/mOV1aBVYKGA [Accessed 6 Feb. 2020].

[5] Kaggle.com. (2020). Novel Corona Virus 2019 Dataset. [online] Available at: https://www.kaggle.com/sudalairajkumar/novel-corona-virus-2019-dataset [Accessed 6 Feb. 2020].

[6] Gisanddata.maps.arcgis.com. (2020). Operations Dashboard for ArcGIS. [online] Available at: https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6 [Accessed 6 Feb. 2020].

[7] Kaggle.com. (2020). SRK | Kaggle. [online] Available at: https://www.kaggle.com/sudalairajkumar [Accessed 6 Feb. 2020].

[8] Plotly. (2020). Modern Analytic Apps for the Enterprise. [online] Available at: https://plot.ly/ [Accessed 6 Feb. 2020].

[9] Cdc.gov. (2020). Prevention, Treatment of Novel Coronavirus (2019-nCoV) | CDC. [online] Available at: https://www.cdc.gov/coronavirus/2019-ncov/about/prevention-treatment.html [Accessed 6 Feb. 2020].

[10] Python-visualization.github.io. (2020). Folium — Folium 0.10.1 documentation. [online] Available at: https://python-visualization.github.io/folium/ [Accessed 6 Feb. 2020].

[11] Anon, (2020). [online] Available at: https://jamanetwork.com/journals/jama/fullarticle/2760500 [Accessed 6 Feb. 2020].

[12] Google Maps. (2020). Google Maps. [online] Available at: https://www.google.com/maps [Accessed 6 Feb. 2020].

[13] Who.int. (2020). WHO | What is a pandemic?. [online] Available at: https://www.who.int/csr/disease/swineflu/frequently_asked_questions/pandemic/en/ [Accessed 6 Feb. 2020].

[14] Science | AAAS. (2020). ‘This beast is moving very fast.’ Will the new coronavirus be contained—or go pandemic?. [online] Available at: https://www.sciencemag.org/news/2020/02/beast-moving-very-fast-will-new-coronavirus-be-contained-or-go-pandemic [Accessed 6 Feb. 2020].

[15] Meera Senthilingam, C. (2020). WHO says Wuhan coronavirus outbreak is not yet a pandemic. [online] CNN. Available at: https://www.cnn.com/2020/02/04/health/who-coronavirus-not-pandemic/index.html [Accessed 6 Feb. 2020].